In [286]:
import re 

In [287]:
symbol_set = {'>' : '⊃',
              '&' : '·',
              '=' : '≡'}

connectives = {'⊃', '·', '≡', 'v'}

In [288]:
def drop_parentheses(s):
    return s.replace('(', '').replace(')', '')


def get_literals(premiss):
    l = []
    for i, c in enumerate(premiss):
        if c.isalpha() and c not in connectives:
            if premiss[i - 1] != '~':
                l.append(c)
            else:
                l.append('~' + c)

    return l[0], l[1]

def equiv_to_cnf(fl, ll):
    return f"({fl} v ~{ll}) · ({ll} v ~{fl})"


def apply_deMorgan(premiss, mc):
    
    literals = get_literals(premiss)
    literals = ['~' + l if '~' not in l else l.replace('~', '') for l in literals]
    new_mc = 'v' if mc == '·' else '·'
    
    return f"{literals[0]} {new_mc} {literals[1]}"

In [299]:
def extract_clauses(p1, p2, mc):
    
    p1_mc = get_mc(p1)
    p2_mc = get_mc(p2)
    
    clauses = []
    
    if mc == '·':
        
        if '~(' in p1:
            p1 = apply_deMorgan(p1, p1_mc)
        
        if '~(' in p2:
            p2 = apply_deMorgan(p2, p2_mc)
            
        if get_mc(p1) == '·':
            clauses.extend(get_literals(p1))
        else:
            clauses.extend(p1)
            
        if get_mc(p2) == '·':
            clauses.extend(get_literals(p2))
        else:
            clauses.append(p2)
            
        
    # Distribution
    if p1_mc == 'v':
        if p2_mc == '·':
            return drop_parentheses(p1) + ' v ' + get_literals(p2)[0], drop_parentheses(p1) + ' v ' + get_literals(p2)[1]

    return clauses


def get_mc(premiss, m=False):
    
    if m == True:
        for i in range(premiss.index(')') + 1, premiss.rindex('(')):
            if premiss[i] != ' ': return premiss[i]
    else:
        for c in premiss:
            if c in connectives: return c
           

def convert_to_cnf(premiss, is_conclusion=False):
    
    for key in symbol_set:
        premiss = premiss.replace(key, symbol_set[key])
    
    if premiss.count(')') == 2:
       mc = get_mc(premiss, m=True)
       
       if mc == 'v' or mc == '·':
            clauses = extract_clauses(premiss[0 : premiss.index(')') + 1], premiss[premiss.rindex('(') - 1 : ], mc)
            return clauses
            
    elif '⊃' in premiss:
        premiss = '~' + premiss
        premiss = premiss.replace('⊃', 'v')        
    
    elif '≡' in premiss:
        first_letter, last_letter = get_literals(premiss)
        premiss = f"({first_letter} · {last_letter}) v (~{first_letter} · ~{last_letter})"
        cnf = equiv_to_cnf(first_letter, last_letter)
    
    if is_conclusion == True:
        negated_conclusion = f"~({premiss})"
        if '~(' in negated_conclusion:
            negated_conclusion = apply_deMorgan(negated_conclusion, get_mc(negated_conclusion))
        
        return negated_conclusion
    
    
def construct_proof(clauses):
    
    for i, c in enumerate(clauses):
        print(f"{i + 1} {c}")
    

In [300]:
argument = ['(L v M) · (N · O)',
            '(~L · O) · ~(~L · M)',
            '~L · N']

clauses = []

for i, p in enumerate(argument):
    
    cnf = convert_to_cnf(p, is_conclusion = (i==len(argument)-1)) 
    if type(cnf) == str: clauses.append(cnf)
    else: clauses.extend(cnf)

construct_proof(clauses)

1 L v M v N
2 L v M v O
3 ~L
4 O
5 L v ~M
6 L v ~N
